# Disproving A Conjecture About Means of Sets of Prime Powers

Here's a conjecture:

Consider the set of all powers of a prime, from $1$ to $p^k$: $P = \{p^0, p^1, p^2, ... p^k\}$.
Pull out two subsets, $S$ and $T$ that have no common elements.

For example, let $p = 2$ and $k = 10$, so $P = \{1, 2, 4, 8, ..., 1024\}$. 
Choose the subsets $S = \{1, 2, 8\}$ and $T = \{4, 16, 32\}$.

Observe that $\mu(S) = 11/3 \neq 52/3 = \mu(T)$

The conjecture is that this will always be true.

For any set of prime powers: $P = \{p^0, ..., p^k\}$, if $S, T \subset P, S \cap T = \emptyset$, then
$\mu(S) \neq \mu(T)$

## Re-Phrasing the Conjecture

Let's test this, for $p = 2$a

First, *every* integer is a unique sum of powers of two.

- 17 = 16 + 1
- 7 = 4 + 2 + 1
- 11 = 8 + 2 + 1
- 52 = 32 + 16 + 4

and so on.

You can read that sum right from the number's binary representation.

- $17 = 10001_2$
- $ 7 = 111_2$
- $11 = 1011_2$
- $52 = 110100_2$
...

This means that each distinct subset, $S$, sums to a unique integer: $N(S) = \sum_{p^k \in S} p^k$ 

Also, the number of elements in the subset is just the number of 1-bits in its base-2 representation.

Thus, if $S \subset P$, then $\mu(S) = N(S)/$(1-bits in $N(S)$)

Next, I'll make up a term: I'll call the inverse of this -- (# of 1-bits in the binary representation /  N) -- the 1's density. I'm not crazy about this term and would love a better one. but let me use it for a few minutes.

It does sort of correspond to your intuition. If a number is big but it doesn't have very many 1-bits in its binary representation, then the 1's density is low. If a number's little, but its binary representation is full of 1's, then the 1's density is high.

For example,

- $512 = 1000000000_2$ -- low 1's density
- $15 = 1111_2$ -- high 1's density.
- $3 = 11_2$ -- even higher 1's density


No, wait. That's not right. I'm claiming that as long as they don't have any of the *same* one bits set, they won't have the same 1's density.

This suggests a more grandiose conjecture: *No* two integers ever have the same 1's density.

Mmmm ... No. That's easy to disprove.

When I look at, say, the first 100 integers, 69 and 92 have the same 1's density.

69 has 3 one-bits, 92 has 4 one-bits. (3/69= 1/23 = 4/92).  

But they do share bits!

$69 = 1000101_2$ and $92 = 1011100_2$, so they share both $64 = 2^6$ and $4 = 2^2$.

In other words, they correspond to subsets of P that intersect, so the more grandiose conjecture's false,
but I still need to look for counterexamples to the original.

Here that is in pseudo-code (for $p = 2$), rephrased in this new way:

```
if m != n and !share_bits(m, n) then ones_density(m) != ones_density(n).
```

Or, even more briefly:
```
if ones_density(m) == ones_density(n) then share_bits(m, n)
```

## Hunting a Counter-Example

### Finding More Collisions: integers with the same bit-density.

Time to write some code to look for a counter-example.


Let's begin with the two functions I used in the pseudo-code.

In [ ]:
def ones_density(n):
    """Number of one-bits divided by the number itself."""
    return (n).bit_count()/n

ones_density(3)

Here, I'm using a feature, new in Python 3.10, [`int.bit_count()`](https://docs.python.org/3/library/stdtypes.html?highlight=bit_count#int.bit_count), which counts the number of 1-bits in an int.

In [ ]:
def share_bits(m, n):
    """True iff m and n share a bit."""
    return bool(m&n)
print(share_bits(1, 3))
print(share_bits(8, 4))

Next, collect the `ones_density` of all the numbers from 1 to lim-1.

In [ ]:
def densities(lim):
    """Return densities of all numbers from 1 up to but not including lim."""
    d = {}
    for n in range(1, lim):
        d[n] = ones_density(n)
    return d

densities(5)

How many densities are there in all the numbers from 1 up to (but not including) 99?

In [ ]:
d_100 = densities(100)
len(set(d_100.values()))

If every integer had a different density, there would have been 99 values. There are only 98, so some value is there twice -- there's a collision. Where is it?

Let's start out by reversing the mapping -- make a dictionary with the integer as the value and its ones density as its key. Then, every time we look at a new integer, we can see whether we've already seen an integer with that ones value.

If we have, then we report the collision.

In [ ]:
def map_densities_to_ints(lim):
    """Construct density->int map."""
    int_val = {}
    k = int_val.keys()
    for n in range(1, lim):
        d = ones_density(n)
        if d in k:
            print(f"collision: {int_val[d]}, {n}") # the current n, and the first integer with the same ones_density
        else:
            int_val[d] = n
    return int_val

iv_1000 = map_densities_to_ints(200)

It's worth pointing out that `k`, the keys for `int_val`, is a *dictionary view*: not a copy of the keys, but a read-only view that changes as elements are added. It's more like a frozenset than a list, so the `if d in k` test is quite fast, even if k is large.

### Screening the Collisions for Counter-Examples

So we have plenty of collisions.

Now we just have to ask whether colliders share bits.

Let's squirrel away the collisions, then go back and look.

In [ ]:
from collections import defaultdict
def find_collisions(lim):
    """Examine every integer 1..lim-1 for ones_density collisions with other ints in range."""
    int_val = {}
    collisions = defaultdict(set)
    k = int_val.keys()
    for n in range(1, lim):
        d = ones_density(n)
        if d in k:
            c = {n, int_val[d]} # set of the colliding pair
            collisions[d] |= c  # union with the existing set of collisions for that ones_density
        else:
            int_val[d] = n
    return collisions

find_collisions(100)

The conjecture is that collisions are only possible for numbers that have one or more bits in common.
Does this pair share any bits?

In [ ]:
share_bits(69, 92)

Oh? And what bits would those be?

In [ ]:
def show_collision(i, j):
    """Display a collision."""
    print(f"{(i).bit_count()}/{i} == {ones_density(i)} == {(j).bit_count()}/{j}")
    print(f"collision: {i}({i:b}) & {j}({j:b}) == {i&j:b}")

show_collision(69, 92)

That's one case. We need a function that will look at all pairs from each collision set.

In [ ]:
def collisions_share_bits(s):
    """Return true iff all elements in s share bits with one another."""
    l = list(s)   # turn it into a list
    while l:
        end = l.pop()   # pop off the rightmost element
        pairwise_shares = [share_bits(end, elem) for elem in l] # check whether all remaining share bits with end
        if not all(pairwise_shares):
            return False
    return True
print(collisions_share_bits({69, 92}))
print(collisions_share_bits({4, 8}))
print(collisions_share_bits({69, 4, 92, 8}))

That's okay, but really we'd like more detail. 

When there are two elements in a collision set, we only need to ask whether they share bits, but if there are three or more, we want to ask which pairs in the set (if any) don't share bits.

We want to know *every* true exception, so we can go back and look at them.

In [ ]:
from itertools import combinations
def true_exceptions(s):
    "Return set of collisions that don't share bits."
    exceptions = set()
    for pair in combinations(s, 2):
        if not share_bits(*pair):
            exceptions.add(pair)
    return exceptions

print(true_exceptions({69, 92}))
print(true_exceptions({4, 8}))
print(true_exceptions({69, 4, 92, 8}))

In [ ]:
def counter_examples(n):
    """Report any counter examples.
    
    A counterexample is a collision set in the range that has non-colliding members.
    """
    counter_pairs = {}
    for density, collisions in find_collisions(n).items():
        if problems := true_exceptions(collisions):   # Found counterexamples
            for problem in problems:
                counter_pairs[density] = sorted(problem)  # sort by first element
    return sorted(counter_pairs.items())
            
counter_examples(2**14)

In [ ]:
show_collision(7554, 8813)

Same ones-density, no shared bits. The conjecture's ***FALSE***. Bummer.

On the other hand, there are no counter examples even for sets as large as $P = \{1, 2, 4, ..., 2^{12}, 2^{13} \}$

## Larger Values of $p$

If we use bigger values of $p$, are counter examples rarer, the same, or more frequent?

Binary is still a fine way to enumerate the possible subsets, but $N(S)$, in our calculation of ones density, is now the sum of powers of something other than 2.

For example, $1011_2 = 11$ can still represent $S = \{p^3, p^1, p^0\}$, and the size of the set is still `(11).bit_count == 3`, and $N(S) = 1p^3 + 0p^2 + 1p^1 + 1p^0$,
but $p$ is no longer 2.

If, for example, $p = 5$, then $N(S) = 125 + 5 + 1 = 131$, and ones_density $= 131/3 = 0.0229007634$

Let's re-write `ones_density()` as a function of `i` and `p`,
where `i` is the i^th subset, corresponding to i_2, and `p` is what we plug in to get the series that we sum to get the number.

To get the binary, we can just use f-strings: `f"{i:b}"`

In [ ]:
i = 11
for c in f"{i:b}":
    print(c)

so that N(S) is just this:

In [ ]:
def N(i, p=2):
    if p == 2:
        return i
    
    total = 0
    for c in f"{i:b}":
        total = int(c) + p*total
    return total

N(11, 5)

In [ ]:
def ones_density(i, p=2):
    """Return ones_density for set i, base p."""
    return (i).bit_count()/N(i, p)

print(ones_density(11))
print(ones_density(11, 5))
print(ones_density(69))
print(ones_density(69, 5))

Next, we need to go back and re-write any function that calls `ones_density`, to take p as an additional argument.

In [ ]:
from collections import defaultdict
def find_collisions(lim, p=2):
    """Examine every integer 1..lim-1 for ones_density collisions with other ints in range."""
    int_val = {}
    collisions = defaultdict(set)
    k = int_val.keys()
    for n in range(1, lim):
        d = ones_density(n, p)
        if d in k:
            c = {n, int_val[d]} # set of the colliding pair
            collisions[d] |= c  # union with the existing set of collisions for that ones_density
        else:
            int_val[d] = n
    return collisions

find_collisions(100)

In [ ]:
def counter_examples(n, p=2):
    """Report any counter examples.
    
    A counterexample is a collision set in the range that has members without shared bits.
    The collisions are of ones_denisties, using base p.
    """
    counter_pairs = {}
    for density, collisions in find_collisions(n, p).items():
        if problems := true_exceptions(collisions):   # Found counterexamples
            for problem in problems:
                counter_pairs[density] = sorted(problem)  # sort by first element
    return sorted(counter_pairs.items())

n = 1_000_000
print(f"timing search for counter-examples, n={n:_}, p = 2")
%timeit counter_examples(lim, p=2)
print(f"timing search for counter-examples, n={n:_}, p = 11")
%timeit counter_examples(lim, p=11)
print()
p_values = [2, 3, 5, 7, 11]
print(f"looking for counter-examples for p={p_values}")
for p in p_values:
    print(f"p={p}, {counter_examples(lim, p)}")

Well! That's convenient, and quick. And it looks like counter-examples are only for p=2.  

Of course, perhaps `n` just isn't big enough. We really need a proof (or disproof) of the conjecture for p>2.